In [1]:
import os
import re
import textract
import lib.words as wd

In [3]:
nomi = list(n.strip().lower() for n in open('../extraction/nomi/nomi_asti.txt'))
#nomi = set(n.strip().lower() for n in open('../extraction/nomi/nomi_gnorry.txt'))
#nomi = set(n.strip().lower() for n in open('../extraction/nomi/nomi_italiani.txt'))
cognomi = set(n.strip().lower() for n in open('../extraction/nomi/cognomi.txt'))

In [4]:
nomi = nomi[:200]
v = [r.split('\t')[0] for r in nomi]

In [5]:
v += ['ambrogio', 'paolina', 'irma']

In [ ]:
i = 0
with open ('esempi_extracted/5115047380001.txt', 'r') as f:
    for i,line in enumerate(f):
        if i < 15:
            l = line.strip()
            if l != '':
                words = l.split(' ')
                for word in words:
                    if word.lower() in nomi:
                        print word
                        index = words.index(word)
                        print index
                        print words[index - 1], words[index], words[index + 1]

In [10]:
def extract_names_old(folder):
    for file in os.listdir(folder):
        with open ('../extraction/esempi_extracted/' + file, 'r') as f:
            found = False
            for i,line in enumerate(f):
                if found:
                    break
                l = line.strip()
                if l != '':
                    words = l.split(' ')
                    for word in words:
                        if word.lower() in v:
                            index = words.index(word)
                            if words[index - 1].lower() not in ['corso', 'via', 'viale', 'largo', 'porto']:
                                print file, ' '.join(words)
                                found = True
                                break

In [74]:
def extract_names(dirr, is_pdf=False, max=-1):
    text = ""
    for i,f in enumerate(os.listdir(dirr)):
        if i == max:
            break
        if is_pdf and f[-3:] == 'pdf':
            text = textract.process(dirr + f).strip()
        else: 
            with open (dirr + f, 'r') as f2:
                text = f2.read()
        text = wd.replace_evil_dots_and_underscores(text).replace('\n',' ').replace('  ',' ').replace(',','')       
        found = False
        for i,line in enumerate(wd.split_sents(text)):
            if found:
                break
            l = line.strip()
            if l != '':
                words = l.split(' ')
                for word in words:
                    if word.lower() == 'me':
                        index = words.index(word)
                        print f, ' '.join(words[index+1:index+5])
                        found = True
                        break
        if not found:
            print 'NON TROVATO', '\t\t', f

In [62]:
extract_names('../extraction/files_to_label/', True)

dott LUCIA UCCELLI notaio
dott Ambrogio Romano Notaio
Dottoressa Paolina De Liguori
dottor GIOVANNI MURARO Notaio
dottor VINCENZO CALDERINI Notaio
dottor FABRIZIO PASCUCCI notaio
Alessandra Bianchi notaio in
Avv Lucia Di Lieto
Dottor Roberto Colucci Notaio
dottor DONATO CASCAVILLA Notaio
dottor Maria Luisa Sperandeo
Andrea Guglielmoni Notaio in
dottor TOMMASO GAETA Notaio
PAOLA TOTARO Notaio in
Giovanna Falco notaio in
dottor Michele Colasanto Notaio
dottor FABRIZIO PASCUCCI Notaio
Avv Giovanni Piacitelli Notaio
Dottor Notaio PAOLO MICHELI
Dottor Marco Cavallini Notaio


In [75]:
extract_names('../atti_costitutivi_txt/', False, 500)

5049618730001.txt Fabrizio Figurelli Notaio in
4930258720001.txt Dr Raimondo MALINCONICO Notaio
5003601230001.txt Dott Andrea Coletta Notaio
5060167590001.txt Dottor LUCIANO QUAGGIA Notaio
4985610910001.txt dott PIETRO BERNARDI FABBRANI
4934063060001.txt dottor FILIPPO MARINELLI Notaio
4943577600001.txt Zaira LO BIANCO Notaio
5024081290001.txt Dottor ANGELO CACCETTA Notaio
5005776590001.txt FABIO MONACO notaio in
5051922110001.txt dottor Francesco Figlioli notaio
4919128160001.txt dottor Giovannibattista Musto Notaio
5067992100001.txt Dottssa ORFINA SCROCCO Notaio
5002413830001.txt presso il Dottor MARCO
5044504790001.txt dottssa ANNA LAURA PAOLETTI
5052464430001.txt dottor Luigi BARTOLOMEO Notaio
5121967110001.txt dott Roberto Chiari fu
5090226530001.txt dottor Salvatore Cutrupia notaio
4923509370001.txt dr Pasquale Salvatore Placida
4985596990001.txt dottor GIOVANNI MURARO Notaio
5110749920001.txt Notaio Gianmario ANGELINO iscritto
4928305720001.txt dott Gigino ROLLO Notaio
495489545